In [34]:
# import necessary libraries
from bs4 import BeautifulSoup
import re
import os, csv
import shutil
import urllib
from urllib.request import urlopen
from socket import error as SocketError
import errno
import pickle
import pandas as pd
import httplib2, requests, contextlib
from urllib.parse import urljoin, urlparse
from requests.auth import HTTPBasicAuth, HTTPDigestAuth
from os.path import splitext

In [11]:
#TODO: csv file here
micro_sample_cvs = "micro-sample_MANUAL_Apr17.xlsx"


In [4]:
def write_file(str, file_name):
    """This function writes a string str to a given file name"""
    with open(file_name, "a") as text_file:
        text_file.write(str)
def save_json(file, school):
    with open(file, 'wb') as outfile:
#         json.dump(school, outfile)
        pickle.dump(school, outfile)
#want the obj back
def get_obj_back(file):
    with open(file,'rb') as f:
        var = pickle.load(f)
    return var

In [4]:
def check(url):
    """ Helper function, check if url is a valid list <- our backup plan
    This functions helps to check the url that has service unavailable issues
    Since status code fails to check this."""
    
    try:
        urlopen(url)
        
    except urllib.error.URLError:
        print(url + " :URLError")
        return False
    except urllib.error.HTTPError:
        print(url +' :HTTPError')
        return False
    except SocketError:
        print(url + 'SocketError')
        return False
    return True

def check_url(url):
    """This functions uses the status code to determine if the link is valid. This resolves
    the links that redirects and most cases of authentication problems"""
    code = "[no code collected]"
    if url == "":
        return False
    try:
        r = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        code = r.status_code
        #backup plan for service unavailable issues
        if code == 503:
            return check(url)
        if code < 400:
            return True   
    except:
        pass
    print("Encounter an invalid link: " + str(url) +" ---Error code: " + str(code))
    return False    


In [12]:
df = pd.read_excel(micro_sample_cvs)
school_urls = df["URL"]
school_names = df["School name"]

### Get all the links of a websites

In [6]:
def get_children_links(url_parent, hostname, visited, depth):
    """This function recursively gets the children links of a given links"""
    #we have gone through enough levels or visited this link already 
    if depth == 0 or url_parent in visited or not check_url(url_parent):
        return set()
    
    #get the html page
    #parse into a BS object
    html_page = requests.get(url_parent, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
    soup = BeautifulSoup(html_page.text)

    #we visited url_parent, updated into the set
    visited.add(url_parent)
    
    #now checking its children
    for link in soup.findAll('a'):
        #running recursively in a try-except block to prevent broken links break the code
        try:
            pattern = re.compile("((http|ftp)s?://.*?)")
            current_link = link.get('href')
#             print(current_link)
            if not pattern.match(current_link):
                current_link = urljoin(url_parent, current_link)
            
            #check if the link is within the domain (hostname)
            if hostname in current_link:
#                 print(current_link)
                #combine results from its children's links
                get_children_links(current_link, hostname, visited, depth -1)
        except:
            pass
    return visited
#     print(count)

def getLinks(url, depth):
    text = set()
    hostname = urlparse(url).hostname
    return get_children_links(url, hostname, text, depth)
#delete 

### Testing

In [10]:
# test on 1 url
urls = getLinks("https://www.richland2.org/charterhigh/", 5)
count = 0
print("Runing test")
print("Here are all the links from: " + "https://www.richland2.org/charterhigh/" )
for e in urls:
    count += 1
    print(e)
print("There are " + str(count)+ " of them")


Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Runing test
Here are all the links from: https://www.richland2.org/charterhigh/
https://www.richland2.org/CharterHigh/About-Our-School/Calendar
https://www.richland2.org/CharterHigh/School-Information/School-Board
https://www.richland2.org/CharterHigh/
https:/

### Now run this method on 300 sites, save as dictionary and store it as a JSON object

In [ ]:
schools = dict()
cap = len(school_names)
# cap = 2
for i in range(cap):
    link, name = school_urls[i], school_names[i]
    #sanity check make sure all links are valid
    if not check_url(link):
        print("Hey! " + name + " doesnt have a valid link! " + link)
    print(school_names[i])
    #saving school names as keys and list of links as values
    schools.update({name: list(getLinks(link, 4))})
    

RICHLAND TWO CHARTER HIGH
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
Encounter an invalid link: https://www.richland2.org/~ ---Error code: 404
POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL


In [8]:
save_json("list_of_links", schools)

In [6]:
school_list = get_obj_back("list_of_links")
for i in school_list:
    print(i + " has " + str(len(school_list[i])) )
    print(school_list[i])

RICHLAND TWO CHARTER HIGH has 24
['https://www.richland2.org/accessibility.aspx', 'https://www.richland2.org/CharterHigh/About-Us', 'https://www.richland2.org/CharterHigh/School-Information/Directions', 'https://www.richland2.org/CharterHigh/news', 'https://www.richland2.org', 'https://www.richland2.org/CharterHigh/Careers', 'https://www.richland2.org/CharterHigh/', 'https://www.richland2.org/CharterHigh/School-Information/Graduation-2017', 'https://www.richland2.org/CharterHigh/About-Our-School/Calendar', 'https://www.richland2.org/charterhigh/news', 'https://www.richland2.org/charterhigh/About-Our-School/Calendar', 'https://www.richland2.org/CharterHigh/School-Information/Lab-Hours', 'https://www.richland2.org/CharterHigh/News/We-re-Hiring-a-Principal', 'https://www.richland2.org/CharterHigh/School-Information', 'https://www.richland2.org/CharterHigh/Special-Pages/Site-Translations', 'https://www.richland2.org/CharterHigh/Student-Registration', 'https://www.richland2.org/CharterHigh/

Now we can get all the html text by runing requests to get all the content.

### Get all images

Since we all have a list of the links alreay, now we are ready to create folders to store these images

In [79]:
def get_images(url):

    html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
    soup = BeautifulSoup(html_page.text, "lxml")
    images = set()
    # getting the links of the images
    for link in soup.find_all('img'):
        try:
            image = link.get("src")          
            pattern = re.compile("((http|ftp)s?://.*?)")
            #relative link, need to append to parent's link to that the correct link
            if not pattern.match(image):
                image = urljoin(url, image)
            images.add(image)
        except:
            pass

    return images  
def download_image(url, k):
    try:
        response = requests.get(url,stream=True, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        ext = url[url.rindex("."):]
#         print(ext)
#         file_name = url[url.rindex("/")+1: url.rindex("."+ ext)] +"."+ ext
#         print(file_name)
#         if os.path.exists(file_name):
#             file_name = file_name[:file_name.rindex("."+ ext)] +"_1."+ ext
        with open("img" + str(k), 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
            del response
    except:
        print("couldn't get " + url)

### Testing

In [80]:
r  = get_images("http://www.wattslearningcenter.org/#content_main")
k = 1
os.chdir("/Users/anhnguyen/Desktop/research/scraping_Python/tex2")
for i in r:
    print(i)
    download_image(i, k)
    k += 1
#might need to check on size

http://counter.edlio.com/count.jsp?rn=674&i=WLCD-D&s=/index.jsp
https://1.cdn.edl.io/MdD7AmfcxcBFnrzZAUPjnNkl03XISc3hupv1Wh2TtLvb2kOB.jpg
http://www.wattslearningcenter.org/pics/edlio.png
http://www.wattslearningcenter.org/rotating_images/7042/7042_53_1000_0_0.jpg
http://www.wattslearningcenter.org/pics/bullet_arrow.gif
http://www.wattslearningcenter.org/pics/topnav_hover.png


### Get pdfs

In [71]:
def get_pdfs(url):

    html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
    soup = BeautifulSoup(html_page.text, "lxml")
    pdfs = set()
    # getting the links of the images
    for link in soup.findAll('a'):
        try:
            pdf = link.get('href')          
            pattern = re.compile("((http|ftp)s?://.*?)")
            #relative link, need to append to parent's link to that the correct link
            if not pattern.match(pdf):
                pdf = urljoin(url, pdf)
                
            if(pdf.endswith(".pdf")):
                pdfs.add(pdf)
        except:
            pass

    return pdfs 
def download_pdf(url):
    try:
        
        file_name = url[url.rindex("/")+1: url.rindex(".pdf")] +".pdf"
#         print(file_name)
        if os.path.exists(file_name):
            file_name = file_name[:file_name.rindex(".pdf")] +"_1.pdf"
        response = requests.get(url,stream=True, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        with open(file_name, 'wb') as f:
            f.write(response.content)
    except:
        print("couldn't get " + url)

### Testing

In [55]:
m  = get_pdfs("http://inst.eecs.berkeley.edu/~ee16b/sp17/")
os.chdir("/Users/anhnguyen/Desktop/research/scraping_Python/tex2")
for i in m:
    print(i)
    download_pdf(i)

http://inst.eecs.berkeley.edu/~ee16b/sp17/note/circuits/Current.pdf
Current.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/note/linalg/BasisAndDiagonalization.pdf
BasisAndDiagonalization.pdf
https://www.ling.ohio-state.edu/~kbaker/pubs/Singular_Value_Decomposition_Tutorial.pdf
Singular_Value_Decomposition_Tutorial.pdf
couldn't get https://www.ling.ohio-state.edu/~kbaker/pubs/Singular_Value_Decomposition_Tutorial.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/lec/Lecture5B.pdf
Lecture5B.pdf
http://math.mit.edu/~gs/linearalgebra/linearalgebra5_7-1.pdf
linearalgebra5_7-1.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/hw/hw3/sol3.pdf
sol3.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/hw/hw6/sol6.pdf
sol6.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/lec/Lecture11B.pdf
Lecture11B.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/hw/hw6/prob6.pdf
prob6.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/hw/hw5/prob5.pdf
prob5.pdf
http://inst.eecs.berkeley.edu/~ee16b/sp17/lec/Lecture2B.pdf
Lecture2B.pdf
h

Here is the fun part. We will run our code to get everything, from text, images, and pdfs

In [9]:
def format_folder_name (k, name):
    """Format a folder nicely for easy access"""
    if k < 10: # Add two zeros to the folder name if k is less than 10 (for ease of organizing the output folders)
        dirname = "00" + str(k) + " " + name
    elif k < 100: # Add one zero if k is less than 100
        dirname = "0" + str(k) + " " + name
    else: # Add nothing if k>100
        dirname = str(k) + " " + name
    return dirname

   

In [90]:

def run_master(school_list, parent_folder):
    """webscrap links and print output to appropriate folders"""
    #navigate to parent folder
    k = 0
    os.chdir(parent_folder)
    for k, school in enumerate(school_list):
        # create dir my_folder if it doesn't exist yet
        os.chdir(parent_folder)
        curr_folder_name = format_folder_name(k, school)
        if not os.path.exists(curr_folder_name):
            os.makedirs(curr_folder_name)
        #navigate to the correct folder, ready to scrap
        os.chdir(curr_folder_name)
        
        #scrap link to get html content
        for url in school_list[school]:
            html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
            soup = BeautifulSoup(html_page.text)
            try:
                #beautify text name for easy access, accept alphanumeric letters only
                #name is the title of the page
                file_name = re.sub('[^A-Za-z0-9]+', '', soup.title.string)
            except:
                file_name = "no name"
#                 i += 1
            #create folder if not exists & go to that folder
            if not os.path.exists(file_name):
                os.makedirs(file_name)
            os.chdir(file_name)
            
            #format text as follow 
            #first line: url, second line :-------------(separater), third line - end: scraping result
            write_file("URL:" + str(url)+ "\n","text.txt" )
            write_file("----------------------------------------------\n", "text.txt")
            write_file(str(soup.prettify()), "text.txt")
            
            #now get all the images
            list_images = get_images(url)
            for num,img in enumerate(list_images):
                download_image(img, num)
            #now get all the pdfs
            list_pdf = get_pdfs(url)  
            for pdf in list_pdf:
                download_pdf(pdf)
                
            #done, go back to parent directory, ready to scrape the next url
            os.chdir('..')
        
#         k += 1

In [91]:
parent_folder = "/Users/anhnguyen/Desktop/research/scraping_Python/python requests results"
run_master(school_list, parent_folder)

4_Squares_Explained.pdf
LakelandFlyerGHS.pdf
Student-Appeal-Complaint-Policy-Rule-4.26.pdf
LunsfordTinsleyReport2013wCoverLtr.pdf
2015-2016-Polk-State-College-Annual-Equity-Update-Report-Final-Letter.pdf
Response-to-Liberty-Counsel-May-6-2015.pdf
2016-Workplace-Employment-Posters-1.pdf
Annual-Security-Report-2015.pdf
Student-Attainment-by-Race-Ethnicity-2012-2013.pdf
Equal-Access-Equal-Opportunity-Contact-List-20170316.pdf
Transcript-Request-Form-GHS-1.pdf
BartowATC_Campus.pdf
LCHS-Washington-DC-trip-registration-tip-sheet-08-31-2016.pdf
Our-Students-Our-Success-160818.pdf
SACSCOC-Institutional-Requirements-for-Public-Disclosure-2014-15.pdf
2016-daily-crime-log.pdf
Student-Appeal-Complaint-Policy-Rule-4.26.pdf
LunsfordTinsleyReport2013wCoverLtr.pdf
2015-2016-Polk-State-College-Annual-Equity-Update-Report-Final-Letter.pdf
Response-to-Liberty-Counsel-May-6-2015.pdf
2016-Workplace-Employment-Posters-1.pdf
Annual-Security-Report-2015.pdf
Student-Attainment-by-Race-Ethnicity-2012-2013.pdf
E